Skrypt wczytujący obrazki oraz wycinający próbki tekstury:

In [ ]:
from PIL import Image, ImageFilter
import os

def cut_texture(img_folder, save_folder, img_size=(512, 512), sample_size=(128, 128)):
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    # Iteruje przez wszystkie pliki w folderze z obrazami
    for filename in os.listdir(img_folder):
        if filename.endswith('.jpg') or filename.endswith('.png'):
            img_path = os.path.join(img_folder, filename)
            img = Image.open(img_path)

            # Zmiana rozmiaru obrazu
            img = img.resize(img_size, Image.Resampling.LANCZOS)

            # Liczba próbek wzdłuż x i y
            xnum = img.size[0] // sample_size[0]
            ynum = img.size[1] // sample_size[1]

            for i in range(xnum):
                for j in range(ynum):
                    # Wycinanie próbki
                    box = (i * sample_size[0], j * sample_size[1], (i + 1) * sample_size[0], (j + 1) * sample_size[1])
                    sample = img.crop(box)

                    # Zapis próbki do folderu zapisu
                    sample_filename = f"{filename[:-4]}_sample_{i}_{j}.jpg"
                    sample_path = os.path.join(save_folder, sample_filename)
                    sample.save(sample_path)



# Używanie funkcji
img_folder = r"C:\Users\mrchl\OneDrive\Desktop\Programowanie\PWOI\studia_programowanie_w_obliczeniach_inteligentnych\zad3\FOTO3"
save_folder = r"C:\Users\mrchl\OneDrive\Desktop\Programowanie\PWOI\studia_programowanie_w_obliczeniach_inteligentnych\zad3\FOTO3_samples"
cut_texture(img_folder, save_folder, img_size=(512, 512), sample_size=(128, 128))

Skrypt wczytujący próbki tekstury i wyznaczający dla nich cechy tekstury na podstawie modelu zdarzeń. Zbiór wektorów danych zostaje zapisany do pliku csv:

In [3]:
import os
import numpy as np
import pandas as pd
from skimage import io, color, util
from skimage.feature import greycomatrix, greycoprops

def extract_texture_features(img_folders, csv_path):
    # Wartości kierunku i odległości do GLCM
    distances = [1, 3, 5]
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]  # 0, 45, 90, 135 stopni
    properties = ['dissimilarity', 'correlation', 'contrast', 'energy', 'homogeneity', 'ASM']

    # Lista do przechowywania wyników
    rows = []

    # Iteracja przez każdy folder tekstur
    for img_folder in img_folders:
        # Iteracja przez każdy obraz w folderze
        for filename in os.listdir(img_folder):
            if filename.endswith('.jpg') or filename.endswith('.png'):
                img_path = os.path.join(img_folder, filename)
                img = io.imread(img_path, as_gray=True)

                # Zmniejszenie głębi jasności do 5 bitów (64 poziomy)
                img = util.img_as_ubyte(img)
                img = img // 4

                # Obliczanie GLCM i cech
                glcm = greycomatrix(img, distances, angles, levels=64, symmetric=True, normed=True)
                feature_values = [greycoprops(glcm, prop).ravel().mean() for prop in properties]

                # Dodawanie do listy wierszy
                row = [filename] + feature_values
                rows.append(row)

    # Utworzenie DataFrame z listy wierszy
    df = pd.DataFrame(rows, columns=['filename'] + properties)

    # Zapis do pliku csv
    df.to_csv(csv_path, index=False)

# Przykładowe użycie
img_folders = [
    r"C:\Users\mrchl\OneDrive\Desktop\Programowanie\PWOI\studia_programowanie_w_obliczeniach_inteligentnych\zad3\FOTO1_samples",
    r"C:\Users\mrchl\OneDrive\Desktop\Programowanie\PWOI\studia_programowanie_w_obliczeniach_inteligentnych\zad3\FOTO2_samples",
    r"C:\Users\mrchl\OneDrive\Desktop\Programowanie\PWOI\studia_programowanie_w_obliczeniach_inteligentnych\zad3\FOTO3_samples"
]
csv_path = r"C:\Users\mrchl\OneDrive\Desktop\Programowanie\PWOI\studia_programowanie_w_obliczeniach_inteligentnych\zad3\texture_features.csv"
extract_texture_features(img_folders, csv_path)

Skrypt do klasyfikacji wektorów cech z wykorzystaniem algorytmu K nearest neighbors:

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# wczytanie danych
data = pd.read_csv(r"C:\Users\mrchl\OneDrive\Desktop\Programowanie\PWOI\studia_programowanie_w_obliczeniach_inteligentnych\zad3\texture_features.csv")



# etykiety klas
# data['class'] = data['filename'].apply(lambda x: x.split('_')[2])
data['class'] = 'unknown'


# wyodrębnienie wektorów cech
X = data[['dissimilarity', 'correlation', 'contrast', 'energy', 'homogeneity', 'ASM']]

# wyodrębnienie etykiet klas
y = data['class']

# podział na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# skalowanie danych
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# uczenie klasyfikatora
clf = KNeighborsClassifier(n_neighbors=3)
clf.fit(X_train, y_train)

# predykcja na zbiorze testowym
y_pred = clf.predict(X_test)

# obliczenie dokładności
accuracy = accuracy_score(y_test, y_pred)
print(f"Dokładność klasyfikatora: {accuracy*100:.2f}%")
